# Machine Learning Model with Spark ML
In this notebook, we build a linear regression model using Spark ML. This notebook relies on the output from `2-preprocessing-with-pyspark.ipynb`, which is a parquet file `../data/housing_scaled.parquet`. 

In [1]:
# Import SparkSession
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

21/08/08 06:42:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
scaled_df = spark.read.parquet("../data/housing_scaled.parquet")

In [3]:
scaled_df.show(5)

+-----+--------------------+--------------------+
|label|            features|     features_scaled|
+-----+--------------------+--------------------+
|4.526|[129.0,322.0,126....|[0.30623297630686...|
|3.585|[1106.0,2401.0,11...|[2.62553233949916...|
|3.521|[190.0,496.0,177....|[0.45104081781631...|
|3.413|[235.0,558.0,219....|[0.55786627466754...|
|3.422|[280.0,565.0,259....|[0.66469173151877...|
+-----+--------------------+--------------------+
only showing top 5 rows



### Linear Regression Model
We split the data into training and test sets, then train a regularised linear regression model using some naive hyperparameters.

In [4]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

In [5]:
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the data to the model
linearModel = lr.fit(train_data)

21/08/08 06:46:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/08 06:46:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Next, generate some predictions and compare them to the labels.

In [6]:
# Generate predictions
predicted = linearModel.transform(test_data)

# Extract the predictions and the "known" correct labels
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

# Zip `predictions` and `labels` into a list
predictionAndLabel = predictions.zip(labels).collect()

# Print out first 5 instances of `predictionAndLabel` 
predictionAndLabel[:5]

[(1.4542345782162025, 0.14999),
 (1.6495895629633672, 0.175),
 (1.482098957606102, 0.332),
 (1.6123448501897313, 0.346),
 (1.6541552239238302, 0.35)]

In [8]:
sc.stop()